This contains the following datasets:
1. Google
2. Unacast
3. Descartes Lab

This returns the merged file on mobility data

# Updated on 05/11/2020

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

# Importing the FIPS file

In [10]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("../Raw data/fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips.iloc[3142]
fips = fips.reset_index(drop=True)
fips = fips[0:3142]
fips.loc[fips["stfips"] == 11]

,stfips,stname,ctyfips,ctyname,fips
319,11,District of Columbia,1,District of Columbia,11001


# Google File

In [4]:
# Importing the google file from the server
google = pd.read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=a88b56a24e1a1e25")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# selecting only the USA
google = google.loc[google["country_region_code"] == "US"]
google = google.reset_index(drop=True)

# deleting irrelevant columns
google = google.iloc[:, 2:]

# Renaming the columns
cols = google.columns
google.rename(columns={cols[0]:"stname", cols[1]:"ctyname", cols[3]:"retail", cols[4]:"grocery", cols[5]:"parks", cols[6]: "transit", cols[7]: "workplace", cols[8]:"residential"}, inplace=True)
google

,stname,ctyname,date,retail,grocery,parks,transit,workplace,residential
0,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,NaN,NaN,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,NaN,NaN,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,NaN,NaN,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,NaN,NaN,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
229122,Wyoming,Weston County,2020-05-04,NaN,NaN,NaN,NaN,-34.0,NaN
229123,Wyoming,Weston County,2020-05-05,NaN,NaN,NaN,NaN,-34.0,NaN
229124,Wyoming,Weston County,2020-05-06,NaN,NaN,NaN,NaN,-35.0,NaN
229125,Wyoming,Weston County,2020-05-07,NaN,NaN,NaN,NaN,-32.0,NaN


In [6]:
# Reshaping the file.
pivot = pd.pivot_table(google, index=["stname", "ctyname"], columns=["date"], values=google.columns[3:], aggfunc='first')

In [7]:
pivot

grocery                                   \
date                      2020-02-15 2020-02-16 2020-02-17 2020-02-18   
stname  ctyname                                                         
Alabama Autauga County           7.0        1.0        0.0        0.0   
        Baldwin County          12.0        8.0        9.0        8.0   
        Barbour County          -2.0       -4.0      -11.0      -19.0   
        Bibb County             10.0       -2.0        6.0        7.0   
        Blount County            1.0        7.0       -5.0       -9.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County        9.0        7.0        9.0       12.0   
        Teton County            -3.0       -8.0       -6.0       -3.0   
        Uinta County            15.0        3.0       17.0       10.0   
        Washakie County         -9.0        NaN        NaN        NaN   
        Weston County            NaN      -14.0        NaN       -7.0   

                                                                       \
date                      2020-02-19 2020-02-20 2020-02-21 2020-02-22   
stname  ctyname                                                         
Alabama Autauga County           0.0       -3.0       -2.0        1.0   
        Baldwin County           9.0        5.0       13.0       17.0   
        Barbour County          -8.0      -24.0      -10.0        2.0   
        Bibb County             -3.0      -11.0        8.0       -2.0   
        Blount County            3.0        0.0       -2.0        3.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County       -5.0       13.0        5.0       13.0   
        Teton County            -1.0        1.0        5.0        5.0   
        Uinta County             8.0       13.0       22.0       17.0   
        Washakie County          NaN        NaN       -7.0        NaN   
        Weston County           -2.0        0.0       -2.0        NaN   

                                                 ...  workplace             \
date                      2020-02-23 2020-02-24  ... 2020-04-30 2020-05-01   
stname  ctyname                                  ...                         
Alabama Autauga County           3.0       -5.0  ...      -37.0      -34.0   
        Baldwin County          20.0        9.0  ...      -34.0      -35.0   
        Barbour County          -9.0      -18.0  ...      -21.0      -25.0   
        Bibb County             -2.0        6.0  ...      -29.0      -31.0   
        Blount County            6.0       -9.0  ...      -33.0      -33.0   
...                              ...        ...  ...        ...        ...   
Wyoming Sweetwater County       10.0       13.0  ...      -34.0      -31.0   
        Teton County            13.0        4.0  ...      -52.0      -50.0   
        Uinta County            15.0       16.0  ...      -29.0      -26.0   
        Washakie County          NaN        NaN  ...      -29.0      -25.0   
        Weston County            NaN        NaN  ...      -39.0      -32.0   

                                                                       \
date                      2020-05-02 2020-05-03 2020-05-04 2020-05-05   
stname  ctyname                                                         
Alabama Autauga County         -20.0      -29.0      -31.0      -33.0   
        Baldwin County         -20.0      -26.0      -31.0      -30.0   
        Barbour County         -24.0      -16.0      -20.0      -21.0   
        Bibb County            -21.0      -29.0      -29.0      -27.0   
        Blount County          -19.0      -26.0      -28.0      -29.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County      -19.0      -19.0      -25.0      -29.0   
        Teton County           -64.0      -67.0      -46.0      -45.0   
        Uinta County            -5.0      -18.0      -25.0      -25.0   
        Washak

In [8]:
# Changing the column names as per the rule: variable_yyyymmdd
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)
                 
# Reseting the index to get rid of the multi-level indexing
pivot = pivot.reset_index()
#pivot

In [11]:
for i in range(pivot.shape[0]):
    if fips.loc[(fips["stname"] == pivot.iloc[i, 0]) & (fips["ctyname"] == pivot.iloc[i, 1])].empty == True:
        for county in fips.loc[fips["stname"] == pivot.iloc[i, 0], "ctyname"]:
            
            
            if pivot.iloc[i, 0] == "District of Columbia":
                pivot.iloc[i, 1] = "District of Columbia"
                
            
            city_county = ["Baltimore", "St. Louis", "Fairfax", "Richmond", "Roanoke"]
            
            if pivot.iloc[i, 1] in city_county:
                pivot.iloc[i, 1] = pivot.iloc[i, 1] + " city"
            
            if pivot.iloc[i, 1] in county:
                pivot.iloc[i, 1] = county
                break
            if pivot.iloc[i, 1] == "La Salle Parish":
                pivot.iloc[i, 1] = "La Salle County"
            if pivot.iloc[i, 1] == "DoÃ±a Ana County":
                pivot.iloc[i, 1] = "Doña Ana County"

In [12]:
# Generating a google only final data frame
google_only = pd.merge(fips, pivot, on=["stname", "ctyname"], how="left")
google_only

,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,grocery_20200219,...,workplace_20200430,workplace_20200501,workplace_20200502,workplace_20200503,workplace_20200504,workplace_20200505,workplace_20200506,workplace_20200507,workplace_20200508,workplace_20200509
0,1,Alabama,1,Autauga County,1001,7.0,1.0,0.0,0.0,0.0,...,-37.0,-34.0,-20.0,-29.0,-31.0,-33.0,-33.0,-34.0,-34.0,-18.0
1,1,Alabama,3,Baldwin County,1003,12.0,8.0,9.0,8.0,9.0,...,-34.0,-35.0,-20.0,-26.0,-31.0,-30.0,-31.0,-31.0,-32.0,-16.0
2,1,Alabama,5,Barbour County,1005,-2.0,-4.0,-11.0,-19.0,-8.0,...,-21.0,-25.0,-24.0,-16.0,-20.0,-21.0,-21.0,-23.0,-23.0,-15.0
3,1,Alabama,7,Bibb County,1007,10.0,-2.0,6.0,7.0,-3.0,...,-29.0,-31.0,-21.0,-29.0,-29.0,-27.0,-28.0,-25.0,-26.0,-14.0
4,1,Alabama,9,Blount County,1009,1.0,7.0,-5.0,-9.0,3.0,...,-33.0,-33.0,-19.0,-26.0,-28.0,-29.0,-32.0,-31.0,-33.0,-18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,37,Sweetwater County,56037,9.0,7.0,9.0,12.0,-5.0,...,-34.0,-31.0,-19.0,-19.0,-25.0,-29.0,-28.0,-28.0,-29.0,-16.0
3139,56,Wyoming,39,Teton County,56039,-3.0,-8.0,-6.0,-3.0,-1.0,...,-52.0,-50.0,-64.0,-67.0,-46.0,-45.0,-50.0,-47.0,-47.0,-57.0
3140,56,Wyoming,41,Uinta County,56041,15.0,3.0,17.0,10.0,8.0,...,-29.0,-26.0,-5.0,-18.0,-25.0,-25.0,-24.0,-24.0,-23.0,-10.0
3141,56,Wyoming,43,Washakie County,56043,-9.0,NaN,NaN,NaN,NaN,...,-29.0,-25.0,NaN,NaN,-15.0,-25.0,-28.0,-32.0,-30.0,NaN


In [13]:
merged_set = set(google_only.iloc[:, 4].tolist())
pivot_set = set(pivot.iloc[:, 1].tolist())
google_only

,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,grocery_20200219,...,workplace_20200430,workplace_20200501,workplace_20200502,workplace_20200503,workplace_20200504,workplace_20200505,workplace_20200506,workplace_20200507,workplace_20200508,workplace_20200509
0,1,Alabama,1,Autauga County,1001,7.0,1.0,0.0,0.0,0.0,...,-37.0,-34.0,-20.0,-29.0,-31.0,-33.0,-33.0,-34.0,-34.0,-18.0
1,1,Alabama,3,Baldwin County,1003,12.0,8.0,9.0,8.0,9.0,...,-34.0,-35.0,-20.0,-26.0,-31.0,-30.0,-31.0,-31.0,-32.0,-16.0
2,1,Alabama,5,Barbour County,1005,-2.0,-4.0,-11.0,-19.0,-8.0,...,-21.0,-25.0,-24.0,-16.0,-20.0,-21.0,-21.0,-23.0,-23.0,-15.0
3,1,Alabama,7,Bibb County,1007,10.0,-2.0,6.0,7.0,-3.0,...,-29.0,-31.0,-21.0,-29.0,-29.0,-27.0,-28.0,-25.0,-26.0,-14.0
4,1,Alabama,9,Blount County,1009,1.0,7.0,-5.0,-9.0,3.0,...,-33.0,-33.0,-19.0,-26.0,-28.0,-29.0,-32.0,-31.0,-33.0,-18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,37,Sweetwater County,56037,9.0,7.0,9.0,12.0,-5.0,...,-34.0,-31.0,-19.0,-19.0,-25.0,-29.0,-28.0,-28.0,-29.0,-16.0
3139,56,Wyoming,39,Teton County,56039,-3.0,-8.0,-6.0,-3.0,-1.0,...,-52.0,-50.0,-64.0,-67.0,-46.0,-45.0,-50.0,-47.0,-47.0,-57.0
3140,56,Wyoming,41,Uinta County,56041,15.0,3.0,17.0,10.0,8.0,...,-29.0,-26.0,-5.0,-18.0,-25.0,-25.0,-24.0,-24.0,-23.0,-10.0
3141,56,Wyoming,43,Washakie County,56043,-9.0,NaN,NaN,NaN,NaN,...,-29.0,-25.0,NaN,NaN,-15.0,-25.0,-28.0,-32.0,-30.0,NaN


In [15]:
merged = (google_only.iloc[:, 3].tolist())
original = (pivot.iloc[:, 1].tolist())
mismatch = list(set(original) - set(merged))
mismatch


[]

# Unacast data

In [16]:
# Importing the unacast data. This has to be downloaded as a gzip file from the unacast google drive. 
# Hence cannot be taken directly from the internet link.
df = pd.read_csv("../Raw data/sds-v3-full-county.csv")

# renaming the "county_fips" to "fips"
df.rename(columns={"county_fips":"fips"}, inplace=True)

# rearranging the columns for pivoting.
col = [7, 3, 5, 6] + [i for i in range(8, 21)]
df = df.iloc[:, col]
#df

In [17]:
# pivoting
pivot = pd.pivot_table(df, index=["fips"], columns=["date"], values=df.columns[2:], aggfunc='first', dropna=False)

# changing the variable names to variable_date format
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)
                 
# reseting the index to get rid of the multi level indexing
pivot = pivot.reset_index()

#pivot

In [18]:
# Making the unacast only dataset. Standalone and complete
unacast_only = pd.merge(fips, pivot, on=["fips"], how="left")

# Deleting the "county_population_date" and replace it with "county_population_2018"
todel = []
count = 0
for col in unacast_only.columns:
    if col.split("_")[0] == "county" and col.split("_")[1] == "population":
        count = count + 1
        if count == 1:
            unacast_only.rename(columns={col:"county_population_2018"}, inplace=True)
        else:
            del unacast_only[col]
#print(count)
unacast_only


,stfips,stname,ctyfips,ctyname,fips,county_population_2018,covid_20200224,covid_20200225,covid_20200226,covid_20200227,...,weekday_20200424,weekday_20200425,weekday_20200426,weekday_20200427,weekday_20200428,weekday_20200429,weekday_20200430,weekday_20200501,weekday_20200502,weekday_20200503
0,1,Alabama,1,Autauga County,1001,55601.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
1,1,Alabama,3,Baldwin County,1003,218022.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
2,1,Alabama,5,Barbour County,1005,24881.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
3,1,Alabama,7,Bibb County,1007,22400.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
4,1,Alabama,9,Blount County,1009,57840.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037,43051.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
3138,56,Wyoming,39,Teton County,56039,23081.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
3139,56,Wyoming,41,Uinta County,56041,20299.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0
3140,56,Wyoming,43,Washakie County,56043,7885.0,PRE,PRE,PRE,PRE,...,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0


# Descartes Lab data

In [19]:
# Importing the descartes lab data
sample = pd.read_csv("https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv")

# Removing unnecessary columns
col = [0, 3, 4, 5, 6, 7, 8]
sample = sample.iloc[:, col]
#sample

In [20]:
# Pivot
pivot = pd.pivot_table(sample, index=["admin1", "admin2"], columns=["date"], values=sample.columns[4:], aggfunc='first', dropna=False)

# Changing the name of the variables as per variable_date
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)

# Reseting the index to remove the multi dimension indexing                 
pivot = pivot.reset_index()

# Renameing the columns stname and ctyname
pivot.rename(columns={"admin1":"stname", "admin2":"ctyname"}, inplace=True)

#pivot

In [21]:
# Descartes data only. Standalone
descartes_only = pd.merge(fips, pivot, on=["stname", "ctyname"], how="left")
#descartes_only

# Merging all the mobility data

In [22]:
unacast_only = unacast_only.iloc[:, 5:]
descartes_only = descartes_only.iloc[:, 5:]

final = pd.concat([google_only, unacast_only, descartes_only], axis=1, join="inner")
final.to_csv("../mobility.csv")

In [23]:
final = pd.read_csv("../mobility.csv")

In [24]:
final

,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,...,samples_20200506,samples_20200507,samples_20200508,samples_20200509,samples_20200510,samples_20200511,samples_20200512,samples_20200513,samples_20200514,samples_20200515
0,0,1,Alabama,1,Autauga County,1001,7.0,1.0,0.0,0.0,...,2351.0,2304.0,2417.0,2280.0,2292.0,2152.0,2380.0,2239.0,2351.0,3309.0
1,1,1,Alabama,3,Baldwin County,1003,12.0,8.0,9.0,8.0,...,9392.0,9516.0,10259.0,10624.0,10702.0,9347.0,10116.0,9196.0,9697.0,15831.0
2,2,1,Alabama,5,Barbour County,1005,-2.0,-4.0,-11.0,-19.0,...,823.0,798.0,839.0,914.0,851.0,755.0,766.0,767.0,811.0,1155.0
3,3,1,Alabama,7,Bibb County,1007,10.0,-2.0,6.0,7.0,...,839.0,809.0,829.0,820.0,845.0,791.0,846.0,809.0,901.0,1172.0
4,4,1,Alabama,9,Blount County,1009,1.0,7.0,-5.0,-9.0,...,2567.0,2488.0,2576.0,2340.0,2442.0,2373.0,2707.0,2440.0,2548.0,3516.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3137,56,Wyoming,35,Sublette County,56035,-21.0,NaN,NaN,NaN,...,896.0,854.0,901.0,893.0,876.0,819.0,864.0,880.0,867.0,1259.0
3138,3138,56,Wyoming,37,Sweetwater County,56037,9.0,7.0,9.0,12.0,...,230.0,258.0,229.0,236.0,209.0,228.0,247.0,205.0,237.0,388.0
3139,3139,56,Wyoming,39,Teton County,56039,-3.0,-8.0,-6.0,-3.0,...,515.0,505.0,530.0,491.0,512.0,496.0,483.0,482.0,494.0,754.0
3140,3140,56,Wyoming,41,Uinta County,56041,15.0,3.0,17.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
final

,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,...,samples_20200430,samples_20200501,samples_20200502,samples_20200503,samples_20200504,samples_20200505,samples_20200506,samples_20200507,samples_20200508,samples_20200509
0,0,1,Alabama,1,Autauga County,1001,7.0,1.0,0.0,0.0,...,2953.0,2979.0,3031.0,3152.0,2290.0,2332.0,2351.0,2304.0,2417.0,2280.0
1,1,1,Alabama,3,Baldwin County,1003,12.0,8.0,9.0,8.0,...,11280.0,11707.0,13624.0,14426.0,9259.0,9281.0,9392.0,9516.0,10259.0,10624.0
2,2,1,Alabama,5,Barbour County,1005,-2.0,-4.0,-11.0,-19.0,...,1053.0,1113.0,1257.0,1173.0,780.0,777.0,823.0,798.0,839.0,914.0
3,3,1,Alabama,7,Bibb County,1007,10.0,-2.0,6.0,7.0,...,1116.0,1040.0,1070.0,1059.0,800.0,841.0,839.0,809.0,829.0,820.0
4,4,1,Alabama,9,Blount County,1009,1.0,7.0,-5.0,-9.0,...,3226.0,3166.0,3310.0,3324.0,2465.0,2531.0,2567.0,2488.0,2576.0,2340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,3143,56,Wyoming,37,Sweetwater County,56037,9.0,7.0,9.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3144,3144,56,Wyoming,39,Teton County,56039,-3.0,-8.0,-6.0,-3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3145,3145,56,Wyoming,41,Uinta County,56041,15.0,3.0,17.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3146,3146,56,Wyoming,43,Washakie County,56043,-9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
final.columns[516]

'county_population_2018'